In [16]:
import cv2
import mediapipe as mp
import pyautogui

In [17]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

mp_drawing = mp.solutions.drawing_utils

screen_width, screen_height = pyautogui.size()

I0000 00:00:1707853948.053228       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2


In [18]:
mp_hands

<module 'mediapipe.python.solutions.hands' from '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/mediapipe/python/solutions/hands.py'>

In [19]:
cap = cv2.VideoCapture(0)
prev_x = None
prev_y = None

In [20]:
while cap.isOpened():
    ret, frame = cap.read()

    # mirror image
    frame = cv2.flip(frame, 1)

    # Convert to rgb --bgr is cv ka default...convert to rgb bcs mediapipe rgb me operate krta hai
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)  # to detect hand landmarks
    if results.multi_hand_landmarks:  # if hands are detected
        for landmarks in results.multi_hand_landmarks:
            # hand check krne ko
            handedness = results.multi_handedness[results.multi_hand_landmarks.index(landmarks)].classification[0].label

            mp_drawing.draw_landmarks(frame, landmarks, mp_hands.HAND_CONNECTIONS)
            # draws hand landmarks and connections on the frame
            index_tip = landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            index_mid = landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP]

            if handedness == "Left":  # left mouse
                
                mcp_x = landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].x
                mcp_y = landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].y

                cursor_x = int(mcp_x * screen_width)
                cursor_y = int(mcp_y * screen_height)

                pyautogui.moveTo(cursor_x, cursor_y, duration=0.1)

                if index_tip.y >= index_mid.y:
                    pyautogui.click()

            elif handedness == "Right":  # right keyboard
                x, y = int(index_tip.x * screen_width), int(index_tip.y * screen_height)
                if prev_x is not None and prev_y is not None:
                    dx = x - prev_x
                    dy = y - prev_y

                    if abs(dx) > abs(dy):
                        if dx > 50:  # right
                            pyautogui.press('right')
                        elif dx < -50:  # left
                            pyautogui.press('left')
                    else:  # Vertical swipe
                        if dy > 50:  # down
                            pyautogui.press('down')
                        elif dy < -50:  # up
                            pyautogui.press('up')

                prev_x = x
                prev_y = y

    cv2.imshow("Gesture Recognition", frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()